In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
using ProgressMeter
using CoordinateTransformations
using MLDataPattern
using JLD2
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyTreeInspector

In [ ]:
import LCPSim
import LearningMPC
import BoxValkyries
import Nets

In [ ]:
# reload("LCPSim")
# reload("LearningMPC")
# reload("BoxValkyries")

In [ ]:
boxval = BoxValkyries.BoxValkyrie()
mechanism = boxval.mechanism
xstar = BoxValkyries.nominal_state(boxval)

basevis = Visualizer()[:box_robot]
setgeometry!(basevis, boxval)
settransform!(basevis[:robot], xstar)

In [ ]:
nx = num_positions(mechanism) + num_velocities(mechanism)
nu = num_velocities(mechanism)

activation = Nets.leaky_relu
function sensitive_loss(λ)
    q = fill(λ, 1, 1 + nx)
    q[1] = 1 - λ
    (params, x, y) -> sum(abs2, 
        q .* (Nets.predict_sensitivity(Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y), x) .- y))
end

p_train = 0.6
p_validate = 0.2

In [ ]:
mpc_params = LearningMPC.MPCParams(
    Δt=0.04,
    horizon=15,
    mip_solver=GurobiSolver(Gurobi.Env(), TimeLimit=120, MIPGap=1e-1, MIPGapAbs=5, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

feet = findbody.(mechanism, ["rf", "lf"])
contacts = [Point3D(default_frame(body), 0., 0, 0) for body in feet]
Q, R = BoxValkyries.default_costs(xstar)
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, contacts, mpc_params.Δt)
lqrsol.S[1,:] .= 0
lqrsol.S[:,1] .= 0
lqrsol.K[:,1] .= 0;

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [-2])
set_configuration!(x0, findjoint(mechanism, "base_rotation"), [π/4])
set_configuration!(x0, findjoint(mechanism, "base_x"), [0.5])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0));

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
env = Gurobi.Env()
results_lqr = LCPSim.simulate(x0, lqrsol, boxval.environment, mpc_params.Δt, 300, GurobiSolver(env, OutputFlag=0));

LearningMPC.playback(basevis[:robot], results_lqr, mpc_params.Δt)

In [ ]:
basename = "box-val-dagger-0.02-v8"
param_file = basename * "-params.jld2"
data_file = basename * ".jld2"

In [ ]:
# params, widths, x_to_u, v_to_y = jldopen(param_file, "r") do file
#     params = file["params"]
#     widths = file["widths"]
#     x_to_u = file["x_to_u"]
#     v_to_y = file["v_to_y"]
#     params, widths, x_to_u, v_to_y
# end;

# train_data, test_data, validation_data = jldopen(data_file, "r") do file
#     train_data = file["train_data"]
#     test_data = file["test_data"]
#     validation_data = file["validation_data"]
#     train_data, test_data, validation_data
# end;

In [ ]:
widths = [nx, 16, 16, 16, 16, nu]
start_params = 0.1 * randn(Nets.Params{Float64}, widths).data;
params = copy(start_params)
x_to_u = AffineMap(eye(nx), zeros(nx))
v_to_y = AffineMap(diagm([max(abs(b.lower), abs(b.upper)) for b in LCPSim.all_effort_bounds(mechanism)]), zeros(num_velocities(mechanism)))
train_data = LearningMPC.Sample{Float64}[]
test_data = LearningMPC.Sample{Float64}[]
validation_data = LearningMPC.Sample{Float64}[]

In [ ]:
net = Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y)

In [ ]:
net_controller = x -> begin
    Nets.predict(net, state_vector(x))
end

last_trajectory = []

mpc_controller = LearningMPC.MPCController(mechanism, boxval.environment, 
    mpc_params, lqrsol, [net_controller, lqrsol]);
mpc_controller.callback = (x, results) -> begin
    if !isnull(results.lcp_updates)
        global last_trajectory
        last_trajectory = get(results.lcp_updates)
        LearningMPC.playback(basevis[:robot], get(results.lcp_updates), mpc_controller.params.Δt)
        if !isnull(results.jacobian)
#             xv = state_vector(x)
#             yJ = hcat(get(results.lcp_updates)[1].input, get(results.jacobian))
            r = rand()
            sample = LearningMPC.Sample(x, results)
            if r < p_train
                push!(train_data, sample)
            elseif r < p_train + p_validate
                push!(validation_data, sample)
            else
                push!(test_data, sample)
            end
        end
    end
end

train_loss = sensitive_loss(0.1)
validate_loss = train_loss
training_losses = Float64[]
validation_losses = Float64[]

p_mpc = 0.2

x_control = MechanismState{Float64}(mechanism)

controller = x -> begin
    set_configuration!(x_control, configuration(x))
    settransform!(basevis[:robot], x_control)
    if rand() < p_mpc
        return mpc_controller(x)
    else
        return net_controller(x)
    end
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
q0 = configuration(xstar) .+ 0.0 .* randn(num_positions(xstar))
# q0[1] += 2
v0 = copy(velocity(xstar)) .+ 0.0 * randn(num_velocities(xstar))
v0[1] = 0.1
set_configuration!(x0, q0)
set_velocity!(x0, v0)
mpc_controller(x0)


In [ ]:
LearningMPC.playback(basevis[:robot], last_trajectory, mpc_params.Δt)

In [ ]:
x0 = MechanismState{Float64}(mechanism)
q0 = configuration(xstar) .+ 0.0 .* randn(num_positions(xstar))
v0 = copy(velocity(xstar)) .+ 0.0 * randn(num_velocities(xstar))
q0[1] = 0.25
v0[1] = -3.0
set_configuration!(x0, q0)
set_velocity!(x0, v0)
solver = GurobiSolver(Gurobi.Env(), OutputFlag=0)
results = LCPSim.simulate(x0, mpc_controller, boxval.environment, mpc_params.Δt, 40, solver);

In [ ]:
LearningMPC.playback(basevis[:robot], results, mpc_params.Δt)

In [199]:
function randomize!(x::MechanismState, xstar::MechanismState)
    q = copy(configuration(xstar))
    q .+= 0.1 .* randn(length(q))
    set_configuration!(x0, q)
    set_velocity!(x0, velocity(xstar) .+ 0.5 .* randn(num_velocities(xstar)))
end

randomize! (generic function with 1 method)

In [ ]:
x0 = MechanismState{Float64}(mechanism)

env = Gurobi.Env()
termination = x -> begin
    (configuration(x)[1] < -1 ||
     configuration(x)[2] < 0.5 || 
     configuration(x)[3] > 1.2 || 
     configuration(x)[3] < -1.2)
end

redirect_stdout(open("training_log.txt", "w")) do
    @showprogress for i in 1:1000
        
        randomize!(x0, xstar)
        results_net = LCPSim.simulate(x0, controller, boxval.environment, mpc_params.Δt, 100, GurobiSolver(env, OutputFlag=0);
            termination=termination)
        Nets.adam!(train_loss, params, shuffleobs(LearningMPC.features.(train_data)), Nets.AdamOpts(learning_rate=0.01 * 0.999^i, batch_size=min(10, length(train_data))))
        push!(training_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), LearningMPC.features.(train_data)))
        push!(validation_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), LearningMPC.features.(validation_data)))

        @show training_losses[end]
        @show validation_losses[end]

        jldopen(data_file, "w") do file
            file["train_data"] = train_data
            file["test_data"] = test_data
            file["validation_data"] = validation_data
        end

        jldopen(param_file, "w") do file
            file["params"] = params
            file["widths"] = widths
            file["x_to_u"] = x_to_u
            file["v_to_y"] = v_to_y
            file["training_losses"] = training_losses
            file["validation_losses"] = validation_losses
        end;
    end 
end

Academic license - for non-commercial use only


Progress:   0%|                                         |  ETA: 11.17 daysWARNING: The specified values for size and/or count will result in 3 unused data points
Progress:   1%|                                         |  ETA: 8 days, 20:45:09WARNING: The specified values for size and/or count will result in 5 unused data points
Progress:   6%|███                                      |  ETA: 6 days, 2:11:10m

In [ ]:
q0 = configuration(xstar) .+ 0.0 .* randn(num_positions(xstar))
# q0[3] = 0.3
v0 = copy(velocity(xstar));
v0[1] = 3.

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_net = LCPSim.simulate(x0, net_controller, boxval.environment, 
    mpc_params.Δt, 300, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

LearningMPC.playback(basevis[:robot], results_net, 0.5 * mpc_params.Δt)

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_lqr = LCPSim.simulate(x0, lqrsol, boxval.environment, 
    mpc_params.Δt, 300, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_lqr[1].state))
settransform!(basevis[:robot], x0)

LearningMPC.playback(basevis[:robot], results_lqr, 0.5 * mpc_params.Δt)

In [ ]:
online_controller = BoxValkyries.OnlineMPCController(boxval, 
    mpc_params, lqrsol, [net_controller, lqr_controller]);

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_online = LCPSim.simulate(x0, online_controller, boxval.environment, mpc_params.Δt, 100, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
playback(basevis[:robot], results_online, mpc_params.Δt)

In [ ]:
using ProfileView
set_configuration!(x0, q0)
set_velocity!(x0, v0)
online_controller(x0)
Profile.clear()
@time @profile online_controller(x0)
ProfileView.view()